<a href="https://colab.research.google.com/github/mihnguyen/udemy-customer-analytics/blob/main/purchase-prediction/notebook_Purchase_Prediction_Pt2_Predictive_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part 2 and a continuation of the [[Udemy CAiP] Purchase Prediction Pt1 - Descriptive Analysis notebook](https://colab.research.google.com/drive/1Acmi4BOuO1V7LCxbnMCVzu46vFSpftA6#scrollTo=97I0svM77BjK&uniqifier=1).
While part 1 is descriptive analysis, part 2 will concern predictive modeling.

###Import Libraries

In [14]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import pickle

from sklearn.linear_model import LogisticRegression

###Prepare Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving purchase data.csv to purchase data.csv


In [3]:
import io
df_purchase = pd.read_csv(io.BytesIO(uploaded['purchase data.csv']))

In [4]:
# Import pickle objects
from google.colab import drive
drive.mount('/content/drive')

import pickle
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks"

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
infile = open(DATA_PATH+'/pca.pickle','rb')
pca = pickle.load(infile)

pca

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [6]:
infile = open(DATA_PATH+'/scaler.pickle','rb')
scaler = pickle.load(infile)

scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
infile = open(DATA_PATH+'/kmeans_pca.pickle','rb')
kmeans_pca = pickle.load(infile)

kmeans_pca

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [8]:
# Standardize data
features = df_purchase[['Sex', 'Marital status', 'Age', 'Education', 'Income', 'Occupation', 'Settlement size']]
df_purchase_segm_std = scaler.transform(features)

# Apply PCA
df_purchase_segm_pca = pca.transform(df_purchase_segm_std)

# Segment data
purchase_segm_kmeans_pca = kmeans_pca.predict(df_purchase_segm_pca)

# Create copy of dataframe
df_purchase_predictors = df_purchase.copy()

# Add segment labels
df_purchase_predictors['Segment'] = purchase_segm_kmeans_pca
segment_dummies = pd.get_dummies(purchase_segm_kmeans_pca, prefix = 'Segment', prefix_sep = '_')
df_purchase_predictors = pd.concat([df_purchase_predictors, segment_dummies], axis = 1)

df_pa = df_purchase_predictors

###Modeling Purchase Probability

Now we will calculate the probability of a customer purchasing at each shop visit. Method used is Logistic Regression.

First, define the dependent and independent variables.

Dependent variable is whether a purchase happens or not (values are either 0 or 1).

As for independent variable, common sense is that price has the most correlation to purchase. So price is the first independent variable. Here let's use the mean price (of all 5 brands) as the independent variable:

In [9]:
Y = df_pa['Incidence']

In [11]:
X = pd.DataFrame()
X['Mean Price'] = (df_pa['Price_1'] +
                   df_pa['Price_2'] +
                   df_pa['Price_3'] +
                   df_pa['Price_4'] +
                   df_pa['Price_5'] ) / 5

Now fit model to predic purchase probability:

In [18]:
model_purchase = LogisticRegression(solver = 'sag') # 'sag' was chosen as the solver as it's optimal for simple problem with large dataset like this one
model_purchase.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

Now check the exact relationship between (Mean) Price (X) and Purchase Probability (Y) using coefficient for the independent variable (coef_ attribute), which is the mean price:

In [19]:
model_purchase.coef_

array([[-2.34834991]])

The coefficient for mean price = -2.35. This indicates that an increase in price will result in a decrease in purchase probability (obviously). However, this coefficient only shows the direction of the relationship (which in this case is reverse, or negative).

Next, let's quantify the magnitude of the negative relationship between mean price and purchase probability.